In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [105]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xlsx',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [106]:
# Data from each sheet can be accessed via key
keyList = list(dfs.keys())

In [107]:
# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Change as China for coordinate search
    dfs[key]=dfs[key].replace({'Country/Region':'Mainland China'}, 'China')
    dfs[key]=dfs[key].replace({'Province/State':'Queensland'}, 'Brisbane')
    dfs[key]=dfs[key].replace({'Province/State':'New South Wales'}, 'Sydney')
    dfs[key]=dfs[key].replace({'Province/State':'Victoria'}, 'Melbourne')
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    dfs[key]['Date_last_updated'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]

In [108]:
# Check 
dfs[keyList[0]].head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_last_updated
0,Hubei,China,01/31/2020 18:00,7153,249,159,2020-01-31 18:00:00
1,Zhejiang,China,01/31/2020 18:00,537,0,11,2020-01-31 18:00:00
2,Guangdong,China,01/31/2020 18:00,436,0,11,2020-01-31 18:00:00
3,Henan,China,01/31/2020 18:00,352,2,3,2020-01-31 18:00:00
4,Hunan,China,01/31/2020 18:00,332,0,2,2020-01-31 18:00:00


In [109]:
# Import data with coordinates (coordinates were called seperately in "Updated_coordinates")
dfs[keyList[0]]=pd.read_csv('{}_data.csv'.format(keyList[0]))

In [110]:
# Save numbers into variables to use in the app
confirmedCases=dfs[keyList[0]]['Confirmed'].sum()
deathsCases=dfs[keyList[0]]['Deaths'].sum()
recoveredCases=dfs[keyList[0]]['Recovered'].sum()

In [111]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Confirmed'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated'][0])
    ChinaList.append(dfTpm['Confirmed'][0])
    OtherList.append(dfTpm['Confirmed'][1:].sum())
    
df_confirmed = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList})    

In [112]:
df_confirmed.head()

,Date,Mainland China,Other locations
0,2020-01-31 18:00:00,11182,146
1,2020-01-31 14:00:00,9783,142
2,2020-01-30 21:30:00,9658,118
3,2020-01-30 11:00:00,8124,111
4,2020-01-29 21:00:00,7678,105


In [113]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Recovered'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Recovered':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Recovered', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated'][0])
    ChinaList.append(dfTpm['Recovered'][0])
    OtherList.append(dfTpm['Recovered'][1:].sum())
    
df_recovered = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList})  

In [114]:
df_recovered.head()

,Date,Mainland China,Other locations
0,2020-01-31 18:00:00,228,8
1,2020-01-31 14:00:00,214,8
2,2020-01-30 21:30:00,179,8
3,2020-01-30 11:00:00,135,8
4,2020-01-29 21:00:00,127,6


In [115]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Deaths'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Deaths':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Deaths', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated'][0])
    ChinaList.append(dfTpm['Deaths'][0])
    OtherList.append(dfTpm['Deaths'][1:].sum())
    
df_deaths = pd.DataFrame({'Date':DateList,
                          'Mainland China':ChinaList,
                          'Other locations':OtherList})  

In [116]:
df_deaths.head()

,Date,Mainland China,Other locations
0,2020-01-31 18:00:00,258,0
1,2020-01-31 14:00:00,213,0
2,2020-01-30 21:30:00,213,0
3,2020-01-30 11:00:00,171,0
4,2020-01-29 21:00:00,170,0


In [117]:
# Save numbers into variables to use in the app
# Change to Sydney time
latestDate=datetime.strftime(df_confirmed['Date'][0] + timedelta(hours=16), '%b %d %Y %H:%M AEDT')
daysOutbreak=(df_confirmed['Date'][0] + timedelta(hours=16) - datetime.strptime('12/31/2019', '%m/%d/%Y')).days

In [118]:
df_confirmed['Date'][0]

Timestamp('2020-01-31 18:00:00')

In [119]:
daysOutbreak

32

In [130]:
# Line plot for confirmed cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_confirmed['Mainland China'].max()+1000, 1000))

# Create empty figure canvas
fig_confirmed = go.Figure()
# Add trace to the figure
fig_confirmed.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Mainland China'],
                         mode='lines+markers',
                         name='Mainland China',
                         line=dict(color='#921113', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Mainland China confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
fig_confirmed.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Other locations'],
                         mode='lines+markers',
                         name='Other Region',
                         line=dict(color='#eb5254', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Other region confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
# Customise layout
fig_confirmed.update_layout(
    title=dict(
        text="<b>Confirmed Cases Timeline<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="Playfair Display")
    ),
    margin=go.layout.Margin(
        l=5,
        r=10,
        b=10,
        t=50,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='#272e3e',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.0f}k'.format(i/1000) for i in tickList]
    ),
    yaxis_title="Total Confirmed Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='#272e3e',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    font=dict(color='#7fafdf')
)

fig_confirmed.show()

In [129]:
# Line plot for recovered cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_recovered['Mainland China'].max()+100, 20))

# Create empty figure canvas
fig_recovered = go.Figure()
# Add trace to the figure
fig_recovered.add_trace(go.Scatter(x=df_recovered['Date'], y=df_recovered['Mainland China'],
                         mode='lines+markers',
                         name='Mainland China',
                         line=dict(color='#0f5525', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_recovered['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Mainland China recovered<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
fig_recovered.add_trace(go.Scatter(x=df_recovered['Date'], y=df_recovered['Other locations'],
                         mode='lines+markers',
                         name='Other Region',
                         line=dict(color='#25d75d', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_recovered['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Other region recovered<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
# Customise layout
fig_recovered.update_layout(
    title=dict(
        text="<b>Recovered Cases Timeline<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="Playfair Display")
    ),
    margin=go.layout.Margin(
        l=5,
        r=10,
        b=10,
        t=50,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='#272e3e',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.0f}'.format(i) for i in tickList]
    ),
    yaxis_title="Total Recovered Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='#272e3e',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    font=dict(color='#7fafdf')
)

fig_recovered.show()

In [131]:
# Line plot for deaths cases
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_deaths['Mainland China'].max()+100, 20))

# Create empty figure canvas
fig_deaths = go.Figure()
# Add trace to the figure
fig_deaths.add_trace(go.Scatter(x=df_deaths['Date'], y=df_deaths['Mainland China'],
                         mode='lines+markers',
                         name='Mainland China',
                         line=dict(color='#fc8715', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_deaths['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Mainland China death<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
fig_deaths.add_trace(go.Scatter(x=df_deaths['Date'], y=df_deaths['Other locations'],
                         mode='lines+markers',
                         name='Other Region',
                         line=dict(color='#fed5ad', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d+timedelta(hours=16), '%b %d %Y %H:%M AEDT') for d in df_deaths['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Other region death<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
# Customise layout
fig_deaths.update_layout(
    title=dict(
        text="<b>Death Cases Timeline<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="Playfair Display")
    ),
    margin=go.layout.Margin(
        l=5,
        r=10,
        b=10,
        t=50,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='#272e3e',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.0f}'.format(i) for i in tickList]
    ),
    yaxis_title="Total Death Case Number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='#272e3e',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    hovermode = 'x',
    legend_orientation="h",
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    font=dict(color='#7fafdf')
)

fig_deaths.show()

In [123]:
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

textList=[]
for area, region in zip(dfs[keyList[0]]['Province/State'], dfs[keyList[0]]['Country/Region']):
    
    if type(area) is str:
        if region == "Hong Kong" or region == "Macau" or region == "Taiwan":
            textList.append(area)
        else:
            textList.append(area+', '+region)
    else:
        textList.append(region)

fig2 = go.Figure(go.Scattermapbox(
        lat=dfs[keyList[0]]['lat'],
        lon=dfs[keyList[0]]['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color='#ff1414',
            size=dfs[keyList[0]]['Confirmed'].tolist(), 
            sizemin=2,
            sizemode='area',
            sizeref=2.*max(dfs[keyList[0]]['Confirmed'].tolist())/(40.**2),
        ),
        text=textList,
        hovertext=['Comfirmed: {}<br>Recovered: {}<br>Death: {}'.format(i, j, k) for i, j, k in zip(dfs[keyList[0]]['Confirmed'],
                                                                                                    dfs[keyList[0]]['Recovered'],
                                                                                                    dfs[keyList[0]]['Deaths'])],
    
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "%{hovertext}<br>" +
                        "<extra></extra>")
    
        )

fig2.update_layout(
    title=dict(
        text="<b>Latest Coronavirus Outbreak Map<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="Playfair Display")
    ),
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=40
    ),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        style="mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=29.538860, 
            lon=173.304781
        ),
        pitch=0,
        zoom=2
    )
)

fig2.show()

In [124]:
import dash
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc

In [222]:
app = dash.Dash(__name__, assets_folder='./assets/',
                meta_tags=[
                    {"name": "viewport", "content": "width=device-width, height=device-height, initial-scale=1.0"}
    ])

In [225]:
app.layout = html.Div(style={'backgroundColor':'#151920'},
    children=[
        html.Div(
            id="header",
            children=[                          
                html.H4(children="Wuhan Coronavirus (2019-nCoV) Outbreak Monitor"),
                html.P(
                    id="description",
                    children="On Dec 31, 2019, the World Health Organization (WHO) was informed of \
                    an outbreak of “pneumonia of unknown cause” detected in Wuhan City, Hubei Province, China – the \
                    seventh-largest city in China with 11 million residents. As of {}, there are over {} cases \
                    of 2019-nCoV confirmed globally.\
                    This dash board is developed to visualise and track the recent reported \
                    cases on a daily timescale.".format(latestDate, confirmedCases),
                ),
                html.P(style={'fontWeight':'bold'},
                    children="Last updated on {}.".format(latestDate))
            ]        
        ),
        html.Div(
            id="number-plate",
            style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.5%'},
                 children=[
                     html.Div(
                         style={'width':'24.4%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'.8%'},
                              children=[html.P(style={'textAlign':'center',
                                                      'fontWeight':'bold','color':'#ffffbf','padding':'1rem'},
                                               children="Days Since Outbreak"),
                                        html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#ffffbf'},
                                               children='{}'.format(daysOutbreak))]),
                     html.Div(style={'width':'24.4%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'.8%'},
                              children=[html.P(style={'textAlign':'center',
                                                      'fontWeight':'bold','color':'#d7191c','padding':'1rem'},
                                               children="Confirmed Cases"),
                                        html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#d7191c'},
                                                children='{}'.format(confirmedCases))]),
                     html.Div(style={'width':'24.4%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'.8%'},
                              children=[html.P(style={'textAlign':'center',
                                                      'fontWeight':'bold','color':'#1a9641','padding':'1rem'},
                                               children="Recovered Cases"),
                                        html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#1a9641'},
                                               children='{}'.format(recoveredCases))]),
                     html.Div(style={'width':'24.4%','backgroundColor':'#272e3e','display':'inline-block','marginTop':'.5%'},
                              children=[html.P(style={'textAlign':'center',
                                                      'fontWeight':'bold','color':'#fdae61','padding':'1rem'},
                                               children="Death Cases"),
                                        html.H3(style={'textAlign':'center',
                                                       'fontWeight':'bold','color':'#fdae61'},
                                                children='{}'.format(deathsCases))])]),
        html.Div(style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.35%','marginTop':'.5%'},
                 children=[
                     html.Div(style={'width':'32.79%','display':'inline-block','marginRight':'.8%'},
                              children=[dcc.Graph(figure=fig_confirmed)]),
                     html.Div(style={'width':'32.79%','display':'inline-block','marginRight':'.8%'},
                              children=[dcc.Graph(figure=fig_recovered)]),
                     html.Div(style={'width':'32.79%','display':'inline-block'},
                              children=[dcc.Graph(figure=fig_deaths)])]),
        html.Div(style={'marginLeft':'1.5%','marginRight':'1.5%','marginBottom':'.5%'},
                 children=[
                     html.Div(style={'width':'100%','display':'inline-block'},
                              children=[dcc.Graph(figure=fig2)]),]),
        html.Div(style={'marginLeft':'1.5%','marginRight':'1.5%'},
                 children=[
                     html.P(style={'textAlign':'center','margin':'auto'},
                            children=["Data source from ", 
                                      html.A('JHU CSSE,', href='https://docs.google.com/spreadsheets/d/1yZv9w9z\
                                      RKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#'),
                                      html.A(' Dingxiangyuan', href='https://ncov.dxy.cn/ncovh5/view/pneumonia?sce\
                                      ne=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0'),
                                      " | 🙏 Pray for China, Pray for the World 🙏 |",
                                      " Developed by ",html.A('Jun', href='https://junye0798.com/')," with ❤️"])])

                         ])


In [226]:
if __name__ == '__main__':
    app.run_server(port=8882)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8882/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2020 16:40:01] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.8.2&m=1580284437 HTTP/1.1